In [58]:
import psycopg2

In [88]:
import json
from dateutil import parser
import time
import os
import subprocess
import pandas as pd



import requests
response = requests.get("https://opensky-network.org/api/states/all")
raw_data = json.loads(response.text)
data = raw_data['states']

In [73]:
#predefined functions to insert the data
def connect(icao24, callsign, origin_country, time_position, last_contact, longitude, latitude, baro_altitude, on_ground, velocity, true_track, vertical_rate, sensors, geo_altitude, squawk, spi, position_source):
    
    con = psycopg2.connect(database="postgres", user="polina@opensky", password="Bigdata2019", host="opensky.postgres.database.azure.com", port="5432")
    cursor = con.cursor()
    query = "INSERT INTO flights_all (icao24, callsign, origin_country, time_position, last_contact, longitude, latitude, baro_altitude, on_ground, velocity, true_track, vertical_rate, sensors, geo_altitude, squawk, spi, position_source, retrieved_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, current_timestamp)"
    cursor.execute(query, (icao24, callsign, origin_country, time_position, last_contact, longitude, latitude, baro_altitude, on_ground, velocity, true_track, vertical_rate, sensors, geo_altitude, squawk, spi, position_source))
    con.commit()
    
    cursor.close()
    con.close()
    return

In [ ]:
def airline_codes(icao, airline, callsign, country):
    
    con = psycopg2.connect(database="postgres", user="polina@opensky", password="Bigdata2019", host="opensky.postgres.database.azure.com", port="5432")
    cursor = con.cursor()
    query = "INSERT INTO airline_codes (icao, airline, callsign, country) VALUES (%s, %s, %s, %s)"
    cursor.execute(query, (icao, airline, callsign, country))
    con.commit()
    
    cursor.close()
    con.close()
    return

In [ ]:
def airline_crashes(operator, place, country):
    
    con = psycopg2.connect(database="postgres", user="polina@opensky", password="Bigdata2019", host="opensky.postgres.database.azure.com", port="5432")
    cursor = con.cursor()
    query = "INSERT INTO airline_crashes (operator, place, country) VALUES (%s, %s, %s)"
    cursor.execute(query, (operator, place, country))
    con.commit()
    
    cursor.close()
    con.close()
    return

In [ ]:
def delete():
    con = psycopg2.connect(database="postgres", user="polina@opensky", password="Bigdata2019", host="opensky.postgres.database.azure.com", port="5432")
    cursor = con.cursor()
    query2 = "with flights as (select callsign from states where on_ground = true) delete from states where id in (select id from states s inner join flights f on f.callsign = s.callsign) or callsign ilike ''"
    cursor.execute(query2)
    con.commit()
    
    cursor.close()
    con.close()
    return

In [89]:
def insert(data):
    for d in range(len(data)):
        if (data[d][2] == 'Germany' or data[d][2] == 'Ukraine' or data[d][2] == 'Moldova' or data[d][2] == 'Kazakhstan'):
            icao24 = data[d][0]
            callsign = data[d][1]
            origin_country = data[d][2]
            time_position = data[d][3]
            last_contact = data[d][4]
            longitude = data[d][5]
            latitude = data[d][6]
            baro_altitude = data[d][7]
            on_ground = data[d][8]
            velocity = data[d][9]
            true_track = data[d][10]
            vertical_rate = data[d][11]
            sensors = data[d][12]
            geo_altitude = data[d][13]
            squawk = data[d][14]
            spi = data[d][15]
            position_source = data[d][16]
            print(origin_country)
        else: continue

        connect(icao24, callsign, origin_country, time_position, last_contact, longitude, latitude, baro_altitude, on_ground, velocity, true_track, vertical_rate, sensors, geo_altitude, squawk, spi, position_source)
        

In [ ]:
insert(data)

In [18]:
#saving data to the Data Frame
data = pd.read_csv("airport_codes.csv", sep=';')

In [ ]:
#iterating through data 
for index, row in data.iterrows():
    icao = row[0]
    airline = row[1]
    callsign = row[2]
    country = row[3]
    
    airline_codes(icao, airline, callsign, country)

In [36]:
crashes = pd.read_csv("airline_crashes.csv", sep=';')

In [ ]:
#splitting the Location column
new = crashes.Location.str.split(", ",expand=True)
crashes["Place"]= new[0] 
crashes["Country"]= new[1] 

In [ ]:
#deleting existing Location column
crashes.drop(columns =["Location"], inplace = True) 

In [ ]:
#replacing missing values
crashes.replace('NaN', None)

In [ ]:
#iterating through Data Frame
for index, row in crashes.iterrows():
    operator = row[0]
    place = row[1]
    country = row[2]
    
    airline_crashes(operator, place, country)